In [5]:
from dataclasses import dataclass, is_dataclass, fields
from typing import get_origin, get_args, Union, Optional, List, Dict, Any


In [6]:

def dict_to_dataclass(data: Dict[str, Any], target_class: type) -> Any:
    """
    Convert a dictionary to a dataclass instance based on type annotations.
    
    Args:
        data: Dictionary containing the data to convert
        target_class: The dataclass type to convert to
    
    Returns:
        An instance of target_class with fields populated from data
    """
    if not is_dataclass(target_class):
        raise ValueError(f"{target_class} is not a dataclass")
    
    if not isinstance(data, dict):
        raise ValueError("Data must be a dictionary")
    
    kwargs = {}
    dataclass_fields = fields(target_class)
    
    for field in dataclass_fields:
        field_name = field.name
        field_type = field.type
        
        if field_name in data:
            field_value = data[field_name]
            converted_value = _convert_value(field_value, field_type)
            kwargs[field_name] = converted_value
        elif field.default is not dataclass.MISSING:
            # Field has a default value, skip it
            continue
        elif field.default_factory is not dataclass.MISSING:
            # Field has a default_factory, skip it
            continue
        else:
            # Check if field is Optional
            if _is_optional(field_type):
                kwargs[field_name] = None
            else:
                raise ValueError(f"Required field '{field_name}' not found in data")
    
    return target_class(**kwargs)


def _convert_value(value: Any, target_type: type) -> Any:
    """Convert a value to the target type."""
    if value is None:
        if _is_optional(target_type):
            return None
        else:
            raise ValueError(f"Cannot convert None to non-optional type {target_type}")
    
    # Handle Optional types
    if _is_optional(target_type):
        # Get the non-None type from Optional[T]
        args = get_args(target_type)
        non_none_types = [arg for arg in args if arg is not type(None)]
        if non_none_types:
            target_type = non_none_types[0]
    
    # Handle Union types (excluding Optional which is Union[T, None])
    origin = get_origin(target_type)
    if origin is Union:
        args = get_args(target_type)
        # Try each type in the union
        for arg_type in args:
            if arg_type is type(None):
                continue
            try:
                return _convert_value(value, arg_type)
            except (ValueError, TypeError):
                continue
        raise ValueError(f"Cannot convert {value} to any type in {target_type}")
    
    # Handle List types
    if origin is list or target_type is list:
        if not isinstance(value, list):
            raise ValueError(f"Expected list, got {type(value)}")
        
        if origin is list:
            list_item_type = get_args(target_type)[0]
            return [_convert_value(item, list_item_type) for item in value]
        else:
            return list(value)
    
    # Handle Dict types
    if origin is dict or target_type is dict:
        if not isinstance(value, dict):
            raise ValueError(f"Expected dict, got {type(value)}")
        
        if origin is dict:
            args = get_args(target_type)
            if len(args) == 2:
                key_type, value_type = args
                return {
                    _convert_value(k, key_type): _convert_value(v, value_type)
                    for k, v in value.items()
                }
        return dict(value)
    
    # Handle dataclass types
    if is_dataclass(target_type):
        if isinstance(value, dict):
            return dict_to_dataclass(value, target_type)
        else:
            raise ValueError(f"Expected dict for dataclass {target_type}, got {type(value)}")
    
    # Handle basic types
    if target_type in (str, int, float, bool):
        if isinstance(value, target_type):
            return value
        
        # Try type conversion
        try:
            if target_type is bool:
                # Handle common boolean representations
                if isinstance(value, str):
                    lower_val = value.lower()
                    if lower_val in ('true', '1', 'yes', 'on'):
                        return True
                    elif lower_val in ('false', '0', 'no', 'off'):
                        return False
                    else:
                        raise ValueError(f"Cannot convert '{value}' to bool")
                else:
                    return bool(value)
            else:
                return target_type(value)
        except (ValueError, TypeError) as e:
            raise ValueError(f"Cannot convert {value} to {target_type}: {e}")
    
    # If no specific handling, try direct assignment
    return value


def _is_optional(type_hint) -> bool:
    """Check if a type hint represents Optional[T] (Union[T, None])."""
    origin = get_origin(type_hint)
    if origin is Union:
        args = get_args(type_hint)
        return len(args) == 2 and type(None) in args
    return False




In [10]:
data = {
    "response":
        {
            "name": "tengo",
            "age": 18
        }
}

@dataclass
class PersonResponse:
    name: str
    age: int
 
@dataclass
class LLMResponse:
    response: PersonResponse


r = dict_to_dataclass(data, LLMResponse)

In [13]:
r.response.name

'tengo'